# Operations on the _n_-Sphere

In [1]:
import sys
sys.path.append("/home/alok/Programming/py-manifolds")

In [2]:
import math

import jax.numpy as jnp
import numpy as np

from manifolds.manifold import ChartPoint, Cotangent, ContravariantTensor, CovariantTensor, Tangent, Tensor
from manifolds.euclidean import EuclideanPoint, IdChart, EuclideanSpace
from manifolds.sphere import Sphere, StereographicChart, SpherePoint

In [3]:
sphere = Sphere(5)  # oddly, don't need to specify a dimension here

In [4]:
northpole = SpherePoint(np.array([0., 0., 5.]))
southpole = SpherePoint(np.array([0., 0., -5.]))

In [5]:
south_projection = sphere.preferred_chart(northpole)
north_projection = sphere.preferred_chart(southpole)

In [6]:
northern_hemi_point = SpherePoint(np.array([0., 3., 4.]))
print(south_projection.point_to_coords(northern_hemi_point))
print(north_projection.point_to_coords(northern_hemi_point))
print(
    south_projection.coords_to_point(
        south_projection.point_to_coords(northern_hemi_point)
    ).coords
)

[0.        1.6666667]
[ 0.       15.000001]
[0.        3.        3.9999998]


In [7]:
t = Tangent(ChartPoint.of_point(northern_hemi_point, south_projection), np.array([1., 1.]))
print(t.to_chart(north_projection))

Tangent(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), v_coords=DeviceArray([ 8.999998, -8.999995], dtype=float32))


In [8]:
ct = Cotangent(ChartPoint.of_point(northern_hemi_point, south_projection), np.array([1., 1.]))
print(ct.to_chart(north_projection))

Cotangent(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), v_coords=(DeviceArray([ 0.11111118, -0.11111112], dtype=float32),))


In [9]:
t_tensor = Tensor(ChartPoint.of_point(northern_hemi_point, south_projection), np.array([1., 2.]), n_contra=0)
print(t_tensor.to_chart(north_projection))

Tensor(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), t_coords=DeviceArray([  8.999998, -17.99999 ], dtype=float32), n_contra=0)


In [10]:
ct_tensor = Tensor(ChartPoint.of_point(northern_hemi_point, south_projection), np.array([3., 1.]), n_contra=1)
print(ct_tensor.to_chart(north_projection))

Tensor(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), t_coords=DeviceArray([ 0.33333352, -0.11111112], dtype=float32), n_contra=1)


In [11]:
t_tensor.tensor_prod(ct_tensor).to_chart(north_projection)

Tensor(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), t_coords=DeviceArray([[ 3.000001 , -6.       ],
             [-0.9999999,  1.999999 ]], dtype=float32), n_contra=1)

In [12]:
t_tensor.to_chart(north_projection).tensor_prod(ct_tensor.to_chart(north_projection))

Tensor(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), t_coords=DeviceArray([[ 3.000001 , -6.       ],
             [-0.9999999,  1.999999 ]], dtype=float32), n_contra=1)

In [13]:
t_tensor.to_chart(north_projection).tensor_prod(t_tensor.to_chart(north_projection))

Tensor(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), t_coords=DeviceArray([[  80.99997, -161.99988],
             [-161.99988,  323.99966]], dtype=float32), n_contra=0)

In [14]:
t_tensor.tensor_prod(t_tensor).to_chart(north_projection)

Tensor(point=ChartPoint(coords=DeviceArray([ 0.      , 14.999996], dtype=float32), chart=StereographicChart(5.0)), t_coords=DeviceArray([[  80.99997, -161.99988],
             [-161.99988,  323.99966]], dtype=float32), n_contra=0)

In [ ]:
def embed_euclidean(pt: SpherePoint):
    return EuclideanPoint(pt.coords)

def euclidean_metric(pt):
    return ContravariantTensor(ChartPoint.of_point(pt, IdChart()), jnp.eye(pt.coords.shape[0]))

north_pole_y = Tangent(ChartPoint.of_point(northpole, south_projection), np.array([0., 1.]))
print(north_pole_y.as_tensor().pushforward(embed_euclidean, IdChart()))

In [16]:
# demonstrate that the metric
metric = euclidean_metric(embed_euclidean(northpole))
print(metric.pullback(embed_euclidean, ChartPoint.of_point(northpole, south_projection)))
metric_other = euclidean_metric(embed_euclidean(northern_hemi_point))
print(metric_other.pullback(embed_euclidean, ChartPoint.of_point(northern_hemi_point, south_projection)))

CovariantTensor(point=ChartPoint(coords=DeviceArray([0., 0., 5.], dtype=float32), chart=<manifolds.euclidean.IdChart object at 0x7f2d8772dfa0>), t_coords=DeviceArray([0., 2., 0.], dtype=float32), n_contra=0)
ContravariantTensor(point=ChartPoint(coords=DeviceArray([0., 0.], dtype=float32), chart=StereographicChart(-5.0)), t_coords=DeviceArray([[4., 0.],
             [0., 4.]], dtype=float32), n_contra=2)
ContravariantTensor(point=ChartPoint(coords=DeviceArray([0.       , 1.6666667], dtype=float32), chart=StereographicChart(-5.0)), t_coords=DeviceArray([[3.2399998, 0.       ],
             [0.       , 3.2399998]], dtype=float32), n_contra=2)
